In [108]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier as KNN
import talib as ta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sm
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from itertools import combinations
from sklearn.linear_model import Ridge

In [109]:
rsiPeriod = 14
adxPeriod = 14
bollingerBandWindow = 20
numberOfFeatures = 11
trainingDataPoints = 3500
window = 60
LE = LabelEncoder()
linReg = LinearRegression()

In [110]:
dataTrain = pd.read_csv("/Users/ishan/Coding/Wpi/StockMarketSimulationIQP/BriannaCode/30y_stock_csvs/GOOG.csv") 
dataTrain.rename(columns = {'CLOSE':'Close'}, inplace = True)
dataTrain.rename(columns = {'HIGH':'High'}, inplace = True)
dataTrain.rename(columns = {'LOW':'Low'}, inplace = True)
dataTrain.rename(columns = {'VOLUME':'Volume'}, inplace = True)
dataTrain.describe()

,OPEN,High,Low,Close,Volume
count,4404.000000,4404.000000,4404.000000,4404.000000,4.404000e+03
mean,677.546245,684.061149,670.982677,677.675074,6.421307e+06
std,632.203442,638.221911,626.386598,632.419201,7.786831e+06
min,49.409801,50.680038,47.800831,49.818268,7.922000e+03
25%,246.616871,249.021606,242.920727,246.508522,1.561050e+06
50%,428.800262,431.425415,425.121536,428.234879,3.734284e+06
75%,972.594971,979.317520,965.272522,972.679977,8.115756e+06
max,3037.270020,3042.000000,2997.750000,3014.179932,8.254163e+07


In [111]:
#import csv
dataTrain['CloseEdited'] = dataTrain['Close'].astype(float)
dataTrain["RSI"] = ta.RSI(dataTrain['CloseEdited'],rsiPeriod).fillna(0)
dataTrain["ADX"] = ta.ADX(dataTrain['High'],dataTrain['Low'],dataTrain['CloseEdited'],adxPeriod).fillna(0)
fastk, fastd = ta.STOCHF(dataTrain['High'],dataTrain['Low'],dataTrain['CloseEdited'])
dataTrain['fastd'] = fastd
dataTrain['fastk'] = fastk
macd, macdsignal, macdhist = ta.MACD(dataTrain['CloseEdited'])
dataTrain['macd'] = macd
dataTrain['macdsignal'] = macdsignal
dataTrain['macdhist'] = macdhist
upper,middle,lower = ta.BBANDS(dataTrain['CloseEdited'])
dataTrain['bb_lowerband'] = lower
dataTrain['bb_middleband'] = middle
dataTrain['bb_upperband'] = upper
newTrain = pd.DataFrame()
newTrain["Close"] = dataTrain['Close'].astype(float)
newTrain["Close"][1:] = newTrain["Close"][0:-1]
newTrain['Volume'] = dataTrain['Volume'].astype(float)
newTrain['RSI'] = dataTrain['RSI'].fillna(0)
newTrain["ADX"] = dataTrain["ADX"].fillna(0)
newTrain['fastd'] = dataTrain['fastd'].fillna(0)
newTrain['fastk'] = dataTrain['fastk'].fillna(0)
newTrain['macd'] = dataTrain['macd'].fillna(0)
newTrain['macdsignal'] = dataTrain['macdsignal'].fillna(0)
newTrain['macdhist'] = dataTrain['macdhist'].fillna(0)
newTrain['bb_lowerband'] = dataTrain['bb_lowerband'].fillna(0)
newTrain['bb_middleband'] = dataTrain['bb_middleband'].fillna(0)
newTrain['bb_upperband'] = dataTrain['bb_upperband'].fillna(0)
trainingSet = newTrain.iloc[:,0:numberOfFeatures].values #convert to numpy to train RNN
trainingSet


array([[4.99826546e+01, 4.48713610e+07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.99826546e+01, 2.29428740e+07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.39527702e+01, 1.83428970e+07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.82906006e+03, 1.65090000e+06, 4.96624008e+01, ...,
        1.79984241e+01, 2.73651742e+03, 2.80489004e+03],
       [2.77205005e+03, 1.93770000e+06, 4.36756107e+01, ...,
        8.23550486e+00, 2.67380973e+03, 2.76934604e+03],
       [2.68260010e+03, 8.76416000e+05, 4.38597538e+01, ...,
        1.95631799e+00, 2.63499861e+03, 2.75056606e+03]])

In [112]:
ySet = dataTrain['Close'].astype(float).values
ySet

array([  49.98265457,   53.95277023,   54.49573517, ..., 2772.05004883,
       2682.60009766, 2684.86010742])

In [113]:
x = newTrain
y = ySet.ravel()
y = LE.fit_transform(y).astype(int)


print(x.shape)
print(y.shape)



knn = KNN(n_neighbors = 4)



sfs1 = SFS(knn, 
           k_features=12, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(x, y)

(4404, 12)
(4404,)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.8s finished

[2022-02-18 14:55:11] Features: 1/12 -- score: 0.25454132606721164[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    1.5s finished

[2022-02-18 14:55:13] Features: 2/12 -- score: 0.25408719346049047[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.4s finished

[2022-02-18 14:55:14] Features: 3/12 -- score: 0.2529518619436876[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done

In [114]:
sfs1.subsets_

{1: {'feature_idx': (4,),
  'cv_scores': array([0.25454133]),
  'avg_score': 0.25454132606721164,
  'feature_names': ('fastd',)},
 2: {'feature_idx': (4, 5),
  'cv_scores': array([0.25408719]),
  'avg_score': 0.25408719346049047,
  'feature_names': ('fastd', 'fastk')},
 3: {'feature_idx': (2, 4, 5),
  'cv_scores': array([0.25295186]),
  'avg_score': 0.2529518619436876,
  'feature_names': ('RSI', 'fastd', 'fastk')},
 4: {'feature_idx': (2, 3, 4, 5),
  'cv_scores': array([0.24659401]),
  'avg_score': 0.24659400544959129,
  'feature_names': ('RSI', 'ADX', 'fastd', 'fastk')},
 5: {'feature_idx': (1, 2, 3, 4, 5),
  'cv_scores': array([0.24477748]),
  'avg_score': 0.24477747502270664,
  'feature_names': ('Volume', 'RSI', 'ADX', 'fastd', 'fastk')},
 6: {'feature_idx': (1, 2, 3, 4, 5, 6),
  'cv_scores': array([0.24477748]),
  'avg_score': 0.24477747502270664,
  'feature_names': ('Volume', 'RSI', 'ADX', 'fastd', 'fastk', 'macd')},
 7: {'feature_idx': (1, 2, 3, 4, 5, 6, 8),
  'cv_scores': array(

In [115]:
sfs1.k_feature_names_

('Close',
 'Volume',
 'RSI',
 'ADX',
 'fastd',
 'fastk',
 'macd',
 'macdsignal',
 'macdhist',
 'bb_lowerband',
 'bb_middleband',
 'bb_upperband')

In [116]:
def best_subset(estimator, X, y, max_size=8, cv=5):
    n_features = X.shape[1]
    subsets = (combinations(range(n_features), k + 1) for k in range(min(n_features, max_size)))

    best_size_subset = []
    for subsets_k in subsets:  # for each list of subsets of the same size
        best_score = -np.inf
        best_subset = None
        for subset in subsets_k: # for each subset
            estimator.fit(X.iloc[:, list(subset)], y)
            # get the subset with the best score among subsets of the same size
            score = estimator.score(X.iloc[:, list(subset)], y)
            if score > best_score:
                best_score, best_subset = score, subset
        # to compare subsets of different sizes we must use CV
        # first store the best subset of each size
        best_size_subset.append(best_subset)
        
    # compare best subsets of each size
    best_score = -np.inf
    best_subset = None
    list_scores = []
    for subset in best_size_subset:
        score = cross_val_score(estimator, X.iloc[:, list(subset)], y, cv=cv).mean()
        list_scores.append(score)
        if score > best_score:
            best_score, best_subset = score, subset

    return best_subset, best_score, best_size_subset, list_scores

In [117]:
best_subsets, best_score, best_size_subset, list_scores = best_subset(linReg,x,y)
best_score

-15.731829932067578

In [118]:
best_subsets

(1, 2, 3, 4, 5, 6, 11)

In [119]:
best_score

-15.731829932067578

In [120]:
best_size_subset

[(9,),
 (1, 11),
 (1, 2, 11),
 (1, 2, 6, 11),
 (1, 2, 3, 6, 11),
 (1, 2, 3, 5, 6, 11),
 (1, 2, 3, 4, 5, 6, 11),
 (1, 2, 3, 4, 5, 6, 7, 11)]

In [121]:
list_scores

[-22.018234335722997,
 -16.26035991305978,
 -15.880082857832216,
 -15.741240176733626,
 -15.74371390969431,
 -15.738710546606754,
 -15.731829932067578,
 -15.75092583141983]